## Libraries

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stat

from math import sqrt
from mlgear.utils import show, display_columns
from surveyweights import normalize_weights


def margin_of_error(n=None, sd=None, p=None, type='proportion', interval_size=0.95):
    z_lookup = {0.8: 1.28, 0.85: 1.44, 0.9: 1.65, 0.95: 1.96, 0.99: 2.58}
    if interval_size not in z_lookup.keys():
        raise ValueError('{} not a valid `interval_size` - must be {}'.format(interval_size,
                                                                              ', '.join(list(z_lookup.keys()))))
    if type == 'proportion':
        se = sqrt(p * (1 - p)) / sqrt(n)
    elif type == 'continuous':
        se = sd / sqrt(n)
    else:
        raise ValueError('{} not a valid `type` - must be proportion or continuous')
    
    z = z_lookup[interval_size]
    return se * z


def print_pct(pct, digits=0):
    pct = pct * 100
    pct = np.round(pct, digits)
    if pct >= 100:
        if digits == 0:
            val = '>99.0%'
        else:
            val = '>99.'
            for d in range(digits - 1):
                val += '9'
            val += '9%'
    elif pct <= 0:
        if digits == 0:
            val = '<0.1%'
        else:
            val = '<0.'
            for d in range(digits - 1):
                val += '0'
            val += '1%'
    else:
        val = '{}%'.format(pct)
    return val


def calc_result(biden_vote, trump_vote, n, interval=0.8):
    GENERAL_POLLING_ERROR = 2.5
    N_SIMS = 100000
    
    biden_moe = margin_of_error(n=n, p=biden_vote/100, interval_size=interval)
    trump_moe = margin_of_error(n=n, p=trump_vote/100, interval_size=interval)
    undecided = (100 - biden_vote - trump_vote) / 2

    biden_mean = biden_vote + undecided * 0.25
    biden_raw_moe = biden_moe * 100
    biden_allocate_undecided = undecided * 0.4
    biden_margin = biden_raw_moe + biden_allocate_undecided + GENERAL_POLLING_ERROR
    
    trump_mean = trump_vote + undecided * 0.25
    trump_raw_moe = trump_moe * 100
    trump_allocate_undecided = undecided * 0.4
    trump_margin = trump_raw_moe + trump_allocate_undecided + GENERAL_POLLING_ERROR
    
    cdf_value = 0.5 + 0.5 * interval
    normed_sigma = stat.norm.ppf(cdf_value)
    
    biden_sigma = biden_margin / 100 / normed_sigma
    biden_sims = np.random.normal(biden_mean / 100, biden_sigma, N_SIMS)
    
    trump_sigma = trump_margin / 100 / normed_sigma
    trump_sims = np.random.normal(trump_mean / 100, trump_sigma, N_SIMS)
    
    chance_pass = np.sum([sim[0] > sim[1] for sim in zip(biden_sims, trump_sims)]) / N_SIMS
    
    low, high = np.percentile(biden_sims - trump_sims, [20, 80]) * 100
    
    return {'mean': biden_mean - trump_mean, 'high': high, 'low': low, 'n': n,
            'raw_moe': biden_raw_moe + trump_raw_moe,
            'margin': (biden_margin + trump_margin) / 2,
            'sigma': (biden_sigma + trump_sigma) / 2,
            'chance_pass': chance_pass}


def print_result(mean, high, low, n, raw_moe, margin, sigma, chance_pass):
    mean = np.round(mean, 1)
    first = np.round(high, 1)
    second = np.round(low, 1)
    sigma = np.round(sigma * 100, 1)
    raw_moe = np.round(raw_moe, 1)
    margin = np.round(margin, 1)
    chance_pass = print_pct(chance_pass, 1)
    if second < first:
        _ = first
        first = second
        second = _
    if second > 100:
        second = 100
    if first < -100:
        first = -100
    print(('Result Biden {} (80% CI: {} to {}) (Weighted N={}) (raw_moe={}pts, margin={}pts, '
           'sigma={}pts) (Biden {} likely to win)').format(mean,
                                                           first,
                                                           second,
                                                           n,
                                                           raw_moe,
                                                           margin,
                                                           sigma,
                                                           chance_pass))
    print('-')
    



def calc_result_sen(dem_vote, rep_vote, n, interval=0.8):
    GENERAL_POLLING_ERROR = 2.5
    N_SIMS = 100000
    
    dem_moe = margin_of_error(n=n, p=dem_vote/100, interval_size=interval)
    rep_moe = margin_of_error(n=n, p=rep_vote/100, interval_size=interval)
    undecided = 100 - dem_vote - rep_vote

    dem_mean = dem_vote + undecided * 0.25
    dem_raw_moe = dem_moe * 100
    dem_allocate_undecided = undecided * 0.4
    dem_margin = dem_raw_moe + dem_allocate_undecided + GENERAL_POLLING_ERROR
    
    rep_mean = rep_vote + undecided * 0.25
    rep_raw_moe = rep_moe * 100
    rep_allocate_undecided = undecided * 0.4
    rep_margin = rep_raw_moe + rep_allocate_undecided + GENERAL_POLLING_ERROR
    
    cdf_value = 0.5 + 0.5 * interval
    normed_sigma = stat.norm.ppf(cdf_value)
    
    dem_sigma = dem_margin / 100 / normed_sigma
    dem_sims = np.random.normal(dem_mean / 100, dem_sigma, N_SIMS)
    
    rep_sigma = rep_margin / 100 / normed_sigma
    rep_sims = np.random.normal(rep_mean / 100, rep_sigma, N_SIMS)
    
    chance_pass = np.sum([sim[0] > sim[1] for sim in zip(dem_sims, rep_sims)]) / N_SIMS
    
    low, high = np.percentile(dem_sims - rep_sims, [20, 80]) * 100
    
    return {'mean': dem_mean - rep_mean, 'high': high, 'low': low, 'n': n,
            'raw_moe': dem_raw_moe + rep_raw_moe,
            'margin': (dem_margin + rep_margin) / 2,
            'sigma': (dem_sigma + rep_sigma) / 2,
            'chance_pass': chance_pass}


def print_result_sen(mean, high, low, n, raw_moe, margin, sigma, chance_pass):
    mean = np.round(mean, 1)
    first = np.round(high, 1)
    second = np.round(low, 1)
    sigma = np.round(sigma * 100, 1)
    raw_moe = np.round(raw_moe, 1)
    margin = np.round(margin, 1)
    chance_pass = print_pct(chance_pass, 1)
    if second < first:
        _ = first
        first = second
        second = _
    if second > 100:
        second = 100
    if first < -100:
        first = -100
    print(('Result Hegar (D) {} (80% CI: {} to {}) (Weighted N={}) (raw_moe={}pts, margin={}pts, '
           'sigma={}pts) (Hegar {} likely to win)').format(mean,
                                                           first,
                                                           second,
                                                           n,
                                                           raw_moe,
                                                           margin,
                                                           sigma,
                                                           chance_pass))
    print('-')

## Load Processed Data

In [2]:
tx_national_survey = pd.read_csv('responses_processed_tx_weighted.csv').fillna('Not presented')
tx_state_survey = pd.read_csv('responses_processed_tx_state_tx_weighted.csv').fillna('Not presented')

/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,40,41,42,43,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Texas Trump-Clinton

In [3]:
options = ['Donald Trump', 'Hillary Clinton', 'Other']
survey_ = tx_national_survey.loc[tx_national_survey['vote2016'].isin(options)].copy()
survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
survey_['vote2016'].value_counts(normalize=True) * survey_.groupby('vote2016')['lv_weight'].mean() * 100

Donald Trump       52.600278
Hillary Clinton    42.809843
Other               4.589878
dtype: float64

In [4]:
options = ['Donald Trump', 'Hillary Clinton', 'Other']
survey_ = tx_state_survey.loc[tx_state_survey['vote2016'].isin(options)].copy()
survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
survey_['vote2016'].value_counts(normalize=True) * survey_.groupby('vote2016')['lv_weight'].mean() * 100

Donald Trump       54.012179
Hillary Clinton    41.487203
Other               4.500618
dtype: float64

## Texas Trump-Biden

In [5]:
options = ['Joe Biden, the Democrat', 'Donald Trump, the Republican', 'Another candidate', 'Not decided']
survey_ = tx_national_survey.loc[tx_national_survey['vote_trump_biden'].isin(options)].copy()
survey_['weight'] = normalize_weights(survey_['weight'])
survey_['rv_weight'] = normalize_weights(survey_['rv_weight'])
survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
survey_['lv_weight_2020'] = normalize_weights(survey_['lv_weight_2020'])
    
print('## NATIONAL TX-WEIGHTED ##')
weighted_n = int(np.round(survey_['weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_trump_biden'].value_counts(normalize=True) * survey_.groupby('vote_trump_biden')['weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result(**calc_result(biden_vote=votes['Joe Biden, the Democrat'],
                           trump_vote=votes['Donald Trump, the Republican'],
                           n=weighted_n))

print('## NATIONAL TX-WEIGHTED + RV ##')
rv_weighted_n = int(np.round(survey_['rv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_trump_biden'].value_counts(normalize=True) * survey_.groupby('vote_trump_biden')['rv_weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result(**calc_result(biden_vote=votes['Joe Biden, the Democrat'],
                           trump_vote=votes['Donald Trump, the Republican'],
                           n=rv_weighted_n))

print('## NATIONAL TX-WEIGHTED + LV ##')
lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_trump_biden'].value_counts(normalize=True) * survey_.groupby('vote_trump_biden')['lv_weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result(**calc_result(biden_vote=votes['Joe Biden, the Democrat'],
                           trump_vote=votes['Donald Trump, the Republican'],
                           n=lv_weighted_n))

print('## NATIONAL TX-WEIGHTED + LV ALT (POST-HOC) ##')
lv_weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_trump_biden'].value_counts(normalize=True) * survey_.groupby('vote_trump_biden')['lv_weight_alt'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result(**calc_result(biden_vote=votes['Joe Biden, the Democrat'],
                           trump_vote=votes['Donald Trump, the Republican'],
                           n=lv_weighted_n))

print('## NATIONAL TX-WEIGHTED + LV 2020 (2020 NATIONAL VOTE MATCH) ##')
lv_weighted_n = int(np.round(survey_['lv_weight_2020'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_trump_biden'].value_counts(normalize=True) * survey_.groupby('vote_trump_biden')['lv_weight_2020'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result(**calc_result(biden_vote=votes['Joe Biden, the Democrat'],
                           trump_vote=votes['Donald Trump, the Republican'],
                           n=lv_weighted_n))


print('## TEXAS TX-WEIGHTED ##')
survey_ = tx_state_survey.loc[tx_state_survey['vote_trump_biden'].isin(options)].copy()
survey_['weight'] = normalize_weights(survey_['weight'])
survey_['rv_weight'] = normalize_weights(survey_['rv_weight'])
survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
survey_['lv_weight_2020'] = normalize_weights(survey_['lv_weight_2020'])

weighted_n = int(np.round(survey_['weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_trump_biden'].value_counts(normalize=True) * survey_.groupby('vote_trump_biden')['weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result(**calc_result(biden_vote=votes['Joe Biden, the Democrat'],
                           trump_vote=votes['Donald Trump, the Republican'],
                           n=weighted_n))

print('## TEXAS TX-WEIGHTED + RV ##')
rv_weighted_n = int(np.round(survey_['rv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_trump_biden'].value_counts(normalize=True) * survey_.groupby('vote_trump_biden')['rv_weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result(**calc_result(biden_vote=votes['Joe Biden, the Democrat'],
                           trump_vote=votes['Donald Trump, the Republican'],
                           n=rv_weighted_n))

print('## TEXAS TX-WEIGHTED + LV ##')
lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_trump_biden'].value_counts(normalize=True) * survey_.groupby('vote_trump_biden')['lv_weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result(**calc_result(biden_vote=votes['Joe Biden, the Democrat'],
                           trump_vote=votes['Donald Trump, the Republican'],
                           n=lv_weighted_n))

print('## TEXAS TX-WEIGHTED + LV ALT (POST-HOC) ##')
lv_weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_trump_biden'].value_counts(normalize=True) * survey_.groupby('vote_trump_biden')['lv_weight_alt'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result(**calc_result(biden_vote=votes['Joe Biden, the Democrat'],
                           trump_vote=votes['Donald Trump, the Republican'],
                           n=lv_weighted_n))

print('## TEXAS TX-WEIGHTED + LV 2020 (2020 NATIONAL VOTE MATCH) ##')
lv_weighted_n = int(np.round(survey_['lv_weight_2020'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_trump_biden'].value_counts(normalize=True) * survey_.groupby('vote_trump_biden')['lv_weight_2020'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result(**calc_result(biden_vote=votes['Joe Biden, the Democrat'],
                           trump_vote=votes['Donald Trump, the Republican'],
                           n=lv_weighted_n))

## NATIONAL TX-WEIGHTED ##
Joe Biden, the Democrat         46.661993
Donald Trump, the Republican    43.759392
Another candidate                3.246609
Not decided                      6.332005
dtype: float64
Result Biden 2.9 (80% CI: -2.5 to 8.4) (Weighted N=1871) (raw_moe=2.9pts, margin=5.9pts, sigma=4.6pts) (Biden 67.5% likely to win)
-
## NATIONAL TX-WEIGHTED + RV ##
Joe Biden, the Democrat         47.607697
Donald Trump, the Republican    44.859859
Another candidate                3.327200
Not decided                      4.205243
dtype: float64
Result Biden 2.7 (80% CI: -2.3 to 7.9) (Weighted N=1828) (raw_moe=3.0pts, margin=5.5pts, sigma=4.3pts) (Biden 67.7% likely to win)
-
## NATIONAL TX-WEIGHTED + LV ##
Joe Biden, the Democrat         47.764399
Donald Trump, the Republican    45.381279
Another candidate                3.315338
Not decided                      3.538984
dtype: float64
Result Biden 2.4 (80% CI: -2.6 to 7.3) (Weighted N=1836) (raw_moe=3.0pts, margin=5.4pts, sigma

In [6]:
options = ['A Democratic candidate', 'A Republican candidate', 'Another candidate', 'Not decided']
survey_ = tx_national_survey.loc[tx_national_survey['vote_senate'].isin(options)].copy()
survey_['weight'] = normalize_weights(survey_['weight'])
survey_['rv_weight'] = normalize_weights(survey_['rv_weight'])
survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
survey_['lv_weight_2020'] = normalize_weights(survey_['lv_weight_2020'])
    
print('## NATIONAL TX-WEIGHTED ##')
weighted_n = int(np.round(survey_['weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_senate'].value_counts(normalize=True) * survey_.groupby('vote_senate')['weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result_sen(**calc_result_sen(dem_vote=votes['A Democratic candidate'],
                                   rep_vote=votes['A Republican candidate'],
                                   n=weighted_n))

print('## NATIONAL TX-WEIGHTED + RV ##')
rv_weighted_n = int(np.round(survey_['rv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_senate'].value_counts(normalize=True) * survey_.groupby('vote_senate')['rv_weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result_sen(**calc_result_sen(dem_vote=votes['A Democratic candidate'],
                                   rep_vote=votes['A Republican candidate'],
                                   n=rv_weighted_n))

print('## NATIONAL TX-WEIGHTED + LV ##')
lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_senate'].value_counts(normalize=True) * survey_.groupby('vote_senate')['lv_weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result_sen(**calc_result_sen(dem_vote=votes['A Democratic candidate'],
                                   rep_vote=votes['A Republican candidate'],
                                   n=lv_weighted_n))

print('## NATIONAL TX-WEIGHTED + LV ALT (POST-HOC) ##')
lv_weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_senate'].value_counts(normalize=True) * survey_.groupby('vote_senate')['lv_weight_alt'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result_sen(**calc_result_sen(dem_vote=votes['A Democratic candidate'],
                                   rep_vote=votes['A Republican candidate'],
                                   n=lv_weighted_n))

print('## NATIONAL TX-WEIGHTED + LV 2020 (2020 NATIONAL VOTE MATCH) ##')
lv_weighted_n = int(np.round(survey_['lv_weight_2020'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_senate'].value_counts(normalize=True) * survey_.groupby('vote_senate')['lv_weight_2020'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result_sen(**calc_result_sen(dem_vote=votes['A Democratic candidate'],
                                   rep_vote=votes['A Republican candidate'],
                                   n=lv_weighted_n))
    
print('## TEXAS TX-WEIGHTED ##')
survey_ = tx_state_survey.loc[tx_state_survey['vote_senate'].isin(options)].copy()
survey_['weight'] = normalize_weights(survey_['weight'])
survey_['rv_weight'] = normalize_weights(survey_['rv_weight'])
survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
survey_['lv_weight_alt'] = normalize_weights(survey_['lv_weight_alt'])
survey_['lv_weight_2020'] = normalize_weights(survey_['lv_weight_2020'])

weighted_n = int(np.round(survey_['weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_senate'].value_counts(normalize=True) * survey_.groupby('vote_senate')['weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result_sen(**calc_result_sen(dem_vote=votes['A Democratic candidate'],
                                   rep_vote=votes['A Republican candidate'],
                                   n=weighted_n))

print('## TEXAS TX-WEIGHTED + RV ##')
rv_weighted_n = int(np.round(survey_['rv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_senate'].value_counts(normalize=True) * survey_.groupby('vote_senate')['rv_weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result_sen(**calc_result_sen(dem_vote=votes['A Democratic candidate'],
                                   rep_vote=votes['A Republican candidate'],
                                   n=rv_weighted_n))

print('## TEXAS TX-WEIGHTED + LV ##')
lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_senate'].value_counts(normalize=True) * survey_.groupby('vote_senate')['lv_weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result_sen(**calc_result_sen(dem_vote=votes['A Democratic candidate'],
                                   rep_vote=votes['A Republican candidate'],
                                   n=lv_weighted_n))

print('## TEXAS TX-WEIGHTED + LV ALT (POST-HOC) ##')
lv_weighted_n = int(np.round(survey_['lv_weight_alt'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_senate'].value_counts(normalize=True) * survey_.groupby('vote_senate')['lv_weight_alt'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result_sen(**calc_result_sen(dem_vote=votes['A Democratic candidate'],
                                   rep_vote=votes['A Republican candidate'],
                                   n=lv_weighted_n))

print('## TEXAS TX-WEIGHTED + LV 2020 (2020 NATIONAL VOTE MATCH) ##')
lv_weighted_n = int(np.round(survey_['lv_weight_2020'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_senate'].value_counts(normalize=True) * survey_.groupby('vote_senate')['lv_weight_2020'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result_sen(**calc_result_sen(dem_vote=votes['A Democratic candidate'],
                                   rep_vote=votes['A Republican candidate'],
                                   n=lv_weighted_n))


## NATIONAL TX-WEIGHTED ##
A Democratic candidate    41.403696
A Republican candidate    42.879888
Another candidate          1.905609
Not decided               13.810807
dtype: float64
Result Hegar (D) -1.5 (80% CI: -11.0 to 8.1) (Weighted N=1699) (raw_moe=3.1pts, margin=10.3pts, sigma=8.1pts) (Hegar 44.9% likely to win)
-
## NATIONAL TX-WEIGHTED + RV ##
A Democratic candidate    42.800674
A Republican candidate    44.825202
Another candidate          1.741037
Not decided               10.633087
dtype: float64
Result Hegar (D) -2.0 (80% CI: -10.4 to 6.3) (Weighted N=1660) (raw_moe=3.1pts, margin=9.0pts, sigma=7.0pts) (Hegar 41.9% likely to win)
-
## NATIONAL TX-WEIGHTED + LV ##
A Democratic candidate    43.298114
A Republican candidate    45.748069
Another candidate          1.788440
Not decided                9.165377
dtype: float64
Result Hegar (D) -2.4 (80% CI: -10.3 to 5.4) (Weighted N=1670) (raw_moe=3.1pts, margin=8.4pts, sigma=6.6pts) (Hegar 39.6% likely to win)
-
## NATIONAL TX